### Обработка данных

In [42]:
import pandas as pd

melb_df = pd.read_csv('data/melb_data_fe.csv')
melb_df.head(2)

,Suburb,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom,...,Longtitude,Regionname,Propertycount,MeanRoomsSquare,AreaRatio,MonthSale,AgeBuilding,WeekdaySale,StreetType,Weekend
0,Abbotsford,2,house,1480000.0,S,Biggin,2016-03-12,2.5,3067,2,...,144.9984,Northern Metropolitan,4019,25.2,-0.231707,3,46,5,St,1
1,Abbotsford,2,house,1035000.0,S,Biggin,2016-04-02,2.5,3067,2,...,144.9934,Northern Metropolitan,4019,15.8,-0.327660,4,116,5,St,1


In [43]:
# в csv не хранятся типы category and datetime
melb_df[['Type', 'Method', 'SellerG', 'CouncilArea', 'Regionname', 'StreetType', 'Weekend', 'WeekdaySale']] = melb_df[['Type', 'Method', 'SellerG', 'CouncilArea', 'Regionname', 'StreetType', 'Weekend', 'WeekdaySale']].astype('category')
melb_df['Date'] = pd.to_datetime(melb_df['Date'])

#### Сортировка значений

* **sort_values()**, где:
    + **by** — имя или список имён столбцов;
    + **axis** — ось, по которой производится сортировка (0 — строки, 1 — столбцы). По умолчанию - строки;
    + **ascending** — сортировка по возрастанию. По умолчанию True;
    + **ignore_index** — создаются ли новые индексы в таблице. По умолчанию False и сохраняет индексы изначальной таблицы;
    + **inplace** — производится ли замена исходной таблицы на отсортированную. По умолчанию False;

In [23]:
melb_df.sort_values( #сортировка по нескольким значениям
    by=['Date', 'AreaRatio'],
    ascending=[True, False],
    ignore_index=True
).loc[:, ['Date', 'AreaRatio']].head(3)

,Date,AreaRatio
0,2016-01-28,0.076923
1,2016-01-28,-0.590909
2,2016-03-09,1.000000


#### Группировка данных

* **groupby()**, где:
    + **by** — имя или список имён столбцов;
    + **axis** — ось, по которой производится группировка (0 — строки, 1 — столбцы). По умолчанию по строкам;
    + **as_index** — добавляется ли дополнительный индекс к таблице. По умолчанию True;

In [39]:
melb_df.groupby(by='Type')['Price'].mean()

Type
house        1.242665e+06
townhouse    9.337351e+05
unit         6.051275e+05
Name: Price, dtype: float64

In [44]:
# минимальное значение расстояния от центра города до объекта в зависимости от его региона
melb_df.groupby('Regionname')['Distance'].min().sort_values(ascending=False)

Regionname
Western Victoria              29.8
Eastern Victoria              25.2
Northern Victoria             21.8
South-Eastern Metropolitan    14.7
Eastern Metropolitan           7.8
Western Metropolitan           4.3
Southern Metropolitan          0.7
Northern Metropolitan          0.0
Name: Distance, dtype: float64

In [49]:
# количество продаж, средние и максимальные цены объектов недвижимости, сгруппированных по номеру месяца продажи.
melb_df.groupby('MonthSale')['Price'].agg(
    ['count', 'mean', 'max', 'nunique']
).sort_values(by='count', ascending=False).head(3)

,count,mean,max,nunique
MonthSale,,,,
8,1850,1.056371e+06,6500000.0,830
7,1835,9.314698e+05,9000000.0,780
5,1644,1.097807e+06,8000000.0,751


In [48]:
# все характеристики внутри группы
melb_df.groupby('Type')['Price'].agg('describe')

,count,mean,std,min,25%,50%,75%,max
Type,,,,,,,,
house,9449.0,1.242665e+06,668078.742092,131000.0,792000.0,1080000.0,1500000.0,9000000.0
townhouse,1114.0,9.337351e+05,395038.245773,300000.0,670000.0,846750.0,1101900.0,3475000.0
unit,3017.0,6.051275e+05,260987.452871,85000.0,440000.0,560000.0,706000.0,3625000.0


In [50]:
# Список уникальных регионов, в которых продавались различные типы домов
melb_df.groupby('Type')['Regionname'].agg(
    		['nunique', set]
)

,nunique,set
Type,,
house,8,"{Western Metropolitan, Eastern Metropolitan, W..."
townhouse,5,"{Western Metropolitan, Eastern Metropolitan, S..."
unit,6,"{Western Metropolitan, Eastern Metropolitan, S..."


In [60]:
# Какой регион имеет наименьшее стандартное отклонение по географической широте (Lattitude)?
melb_df.groupby('Regionname')['Lattitude'].std().sort_values(ascending=True).head(1)

Regionname
Western Victoria    0.011579
Name: Lattitude, dtype: float64

In [82]:
# Какая риелторская компания имеет наименьшую общую выручку за период с 1 мая по 1 сентября (включительно) 2017 года?
melb_df[(melb_df['Date'] >= pd.to_datetime('2017-05-01')) & (melb_df['Date'] <= pd.to_datetime('2017-09-01'))].groupby('SellerG')['Price'].sum().sort_values(ascending=True).index[0]

'LITTLE'

#### Сводные таблицы

* С помощью **groupby()**;

* **pivot_table()**, где:
    + **values** — имя столбца, по которому необходимо получить сводные данные, применяя агрегирующую функцию;
    + **index** — имя столбца, значения которого станут строками сводной таблицы;
    + **columns** — имя столбца, значения которого станут столбцами сводной таблицы;
    + **aggfunc** — имя или список имён агрегирующих функций (по умолчанию — подсчёт среднего, 'mean');
    + **fill_value** — значение, которым необходимо заполнить пропуски (по умолчанию пропуски не заполняются).

In [75]:
# группировка по нескольким колонкам
# результат - Series
melb_df.groupby(['Rooms', 'Type'])['Price'].mean().round().head(9)

Rooms  Type     
1      house         866866.0
       townhouse     592705.0
       unit          389929.0
2      house        1017238.0
       townhouse     710158.0
       unit          610491.0
3      house        1109233.0
       townhouse     984709.0
       unit          850596.0
Name: Price, dtype: float64

In [74]:
# та же группировка, но результат - DataFrame
melb_df.groupby(['Rooms', 'Type'])['Price'].mean().unstack().round().head(3)

Type,house,townhouse,unit
Rooms,,,
1,866866.0,592705.0,389929.0
2,1017238.0,710158.0,610491.0
3,1109233.0,984709.0,850596.0


In [73]:
melb_df.pivot_table(
    values='Price',
    index='Rooms',
    columns='Type',
    fill_value=0
).round().head(3)

Type,house,townhouse,unit
Rooms,,,
1,866866.0,592705.0,389929.0
2,1017238.0,710158.0,610491.0
3,1109233.0,984709.0,850596.0


In [77]:
# площадь участка в зависимости от типа строения и региона
pivot = melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)
pivot

median                        mean              \
Type                        house townhouse unit        house   townhouse   
Regionname                                                                  
Eastern Metropolitan        674.0     233.5  203   717.422847  269.440678   
Eastern Victoria            843.0       0.0  230  3108.960000    0.000000   
Northern Metropolitan       459.5     134.0    0   619.249092  317.325733   
Northern Victoria           724.0       0.0    0  3355.463415    0.000000   
South-Eastern Metropolitan  630.5     240.0  199   664.306701  212.160000   
Southern Metropolitan       586.0     246.0    0   569.643881  278.858824   
Western Metropolitan        531.0     198.0   62   507.883406  244.560669   
Western Victoria            599.5       0.0    0   655.500000    0.000000   

                                        
Type                              unit  
Regionname                              
Eastern Metropolitan        330.444444  
Eastern Victoria            295.333333  
Northern Metropolitan       495.026538  
Northern Victoria             0.000000  
South-Eastern Metropolitan  357.864865  
Southern Metropolitan       466.380245  
Western Metropolitan        557.637232  
Western Victoria              0.000000

In [78]:
pivot.columns # получим все стобцы

MultiIndex([('median',     'house'),
            ('median', 'townhouse'),
            ('median',      'unit'),
            (  'mean',     'house'),
            (  'mean', 'townhouse'),
            (  'mean',      'unit')],
           names=[None, 'Type'])

In [79]:
display(pivot['mean']['townhouse']) # средняя площадь участка по регионам

Regionname
Eastern Metropolitan          269.440678
Eastern Victoria                0.000000
Northern Metropolitan         317.325733
Northern Victoria               0.000000
South-Eastern Metropolitan    212.160000
Southern Metropolitan         278.858824
Western Metropolitan          244.560669
Western Victoria                0.000000
Name: townhouse, dtype: float64

In [84]:
# регионы, в которых средняя площадь здания для домов типа house меньше их медианной площади
mask = pivot['mean']['house'] < pivot['median']['house']
filtered_pivot = pivot[mask]
display(filtered_pivot)
print(list(filtered_pivot.index)) # получаем индексы

median                       mean              \
Type                   house townhouse unit       house   townhouse   
Regionname                                                            
Southern Metropolitan  586.0     246.0    0  569.643881  278.858824   
Western Metropolitan   531.0     198.0   62  507.883406  244.560669   

                                   
Type                         unit  
Regionname                         
Southern Metropolitan  466.380245  
Western Metropolitan   557.637232

['Southern Metropolitan', 'Western Metropolitan']


In [98]:
pivot = melb_df.pivot_table(values='Price', index='Type', columns='SellerG', aggfunc='median')
pivot.loc['unit'].sort_values(ascending=False).index[0]

'Nick'

#### Объединение DataFrame

* **concat()**, где:
    + **objs** — список объектов DataFrame ([df1, df2,…]), которые должны быть сконкатенированы;
    + **axis** — ось определяет направление конкатенации: 0 — конкатенация по строкам (по умолчанию), 1 — конкатенация по столбцам;
    + **join** — либо inner (пересечение), либо outer (объединение);
    + **ignore_index** — по умолчанию False, значения индекса остаются такими, какими они были в исходных данных. При True будут игнорироваться исходные значения и индексы повторно назначаться в последовательном порядке;
<br>
<br>
* **join()** - требует установки ключевого столбца в качестве индекса правой тадлицы, где:
    + **other** — таблица, которую мы присоединяем;
    + **how** — тип объединения ('inner', 'left', 'right', и 'outer'). По умолчанию 'left';
    + **on** — по какому столбцу в «левой» таблице происходит объединение по индексам из «правой»;
    + **lsuffix и rsuffix** — дополнения (суффиксы) к названиям одноимённых столбцов в «левой» и «правой» таблицах;
<br>
<br>
* **merge()**, где:
    + **right** — присоединяемая таблица;
    + **how** — тип объединения. По умолчанию 'inner';
    + **on** — по какому столбцу происходит объединение. Определяется автоматически, но рекомендуется указывать вручную;
    + **left_on** — если названия столбцов в «левой» и «правой» таблицах не совпадают, то отвечает за наименования ключевого столбца исходной таблицы;
    + **right_on** — отвечает за наименование ключевого столбца присоединяемой таблицы;

In [109]:
movies = pd.read_csv('data/movies_data/movies.csv')
dates = pd.read_csv('data/movies_data/dates.csv')
ratings1 = pd.read_csv('data/movies_data/ratings1.csv')
ratings2 = pd.read_csv('data/movies_data/ratings2.csv')
display(movies.head(2))
display(dates.head(2))
display(ratings1.head(2))
display(ratings2.head(2))

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy


,date
0,2000-07-30 18:45:03
1,2000-07-30 18:20:47


,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0


,userId,movieId,rating
0,274,5621,2.0
1,274,5630,3.0


In [110]:
ratings = pd.concat([ratings1, ratings2], ignore_index=True) # объединение таблиц с рейтингами, перестраиваем индексы

ratings = ratings.drop_duplicates(ignore_index=True) # удаляем дубли после объединения, перестраиваем индексы

ratings_dates = pd.concat([ratings, dates], axis=1) # конкатенируем таблицы ratings и dates по столбцам:

display(ratings_dates.head(2))

,userId,movieId,rating,date
0,1,1,4.0,2000-07-30 18:45:03
1,1,3,4.0,2000-07-30 18:20:47


In [113]:
joined = ratings_dates.join( # получим данные фильмов для таблицы рейтингов 
    movies.set_index('movieId'),
    on='movieId',
    how='left'
)
display(joined.head(3))

,userId,movieId,rating,date,title,genres
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller


In [114]:
merged = ratings_dates.merge( # аналогично примеру выше
    movies,
    on='movieId',
    how='left'
)
display(merged.head(3))

,userId,movieId,rating,date,title,genres
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller


In [115]:
# аналогично объединению таблиц
# автоматически удалены дубли
merge_ratings = ratings1.merge(ratings2, how='outer')